In this notebook, I give an example of how to use a custom likelihood function. Use NBViewer to view the notebook [here](https://nbviewer.jupyter.org/github/as4529/gp3/blob/master/examples/lif.ipynb?flush_cache=true). See the accompanying file (lif.py) for the implementation of the likelihood. 

In [1]:
import sys
sys.path.insert(0,'..')
from gp3.inference.mfsvi import MFSVI
from gp3.utils import data as sim
from gp3.kernels.kernels import rbf, inv_softmax, softmax
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
from IPython.display import display
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings('ignore')
from tqdm import trange, tqdm_notebook
import numpy as np
from lif import LIFLike, LIFSim

## Problem Overview and Data Simulation

**Short description of problem**: We want to infer the conditional probability of a neuron firing given stimulation at particular spatial locations. We let $s_i$ denote the "gain" of stimulation at a given location $x_i$. On the right, we assume that we stimulate the neuron at a set of spatial locations $\{x_i\}_{i=1}^N$, and we receive the timing of the first spikes $\{y_i\}_{i=1}^N$. This $y_i$ is some stochastic function of $s_i$. For intuition, note that where the gain of stimulation is higher, the neurons tend to fire sooner.

The inference problem is as follows: given the timings of spikes at a sub-grid of stimulation locations, we want to infer the overall cell shape and quantify our uncertainty.  For inference, we put a GP prior on $\log s$.

$$\log s \sim \mathcal{GP}(\mu(\cdot), K(\cdot, \cdot))$$
$$y_i \sim \text{LIF}(s(x_i))$$

where LIF denotes a leaky integrate and fire likelihood, where the only free parameter is $s$. 

In [2]:
X = sim.sim_X_equispaced(D = 2, N_dim = 20, lower=0, upper=100)
f = sim.sim_f(X, rbf, inv_softmax(np.array([60., 1.])), mu = 6.) - 1e-3*np.sum(np.square(X-50), 1)
ls = np.ones(len(f))
lif_gen = LIFSim(l=ls)
spikes = lif_gen.sim(f)

X_part, y_part = sim.rand_partial_grid(X, spikes, 0.3)
X_full, y_full, obs_idx, imag_idx = sim.fill_grid(X_part, y_part)
color = np.zeros(X_full.shape[0])
color[obs_idx] = 1.0

trace_func = go.Scatter3d(x = X[:,0], y = X[:,1], z=f, mode = 'markers', marker=dict(size = 2), name = "cell shape")
trace_draws = go.Scatter3d(x = X_part[:,0], y = X_part[:,1], z=y_part, mode = 'markers', marker=dict(size = 2), name = "spike times")
fig = tools.make_subplots(rows=1, cols=2, specs=[[{'is_3d': True}, {'is_3d': True}]])
fig.append_trace(trace_func, 1, 1)
fig.append_trace(trace_draws, 1, 2)
iplot(fig)

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]




**More comprehensive description of LIF** (feel free to skip): We use a simplified leaky integrate-and-fire model to characterize the response of a neuron's membrane potential to stimulation at a spatial location $x_i$. We assume

$$ \begin{cases}
 V(t^+)=V_{{\rm reset}} & {\rm if \  cell} {\rm \ spikes \  at \  time} \ t  \\
 \mathrm{d}V(t)=g [V_{{\rm resting}}- V(t)] + l_is(x_i) & {\rm otherwise} \end{cases}$$
where $V_{{\rm reset}}$ is the reset voltage, $V_{{\rm resting}}$ is the resting voltage, and $g$ is the membrane resistance. Here  $l_i s(x_i) $ is the intensity of stimulation that the cell received given a stimulation at $x_i$,  where $l_i$ is the power level of the stimulation and $s(\cdot)$ is the gain of stimulation in space. $s(\cdot)$ is the function we are interested in modeling. Given the membrane potential, we model the firing probability as 
$$ {\rm pr}(\mathrm{d} N(t)=1) =  \psi( V(t)-V_{{\rm th}}),$$

where $\psi$ is a sigmoid function, and $V_{{\rm th}}$ is the spiking threshold. In summary, the LIF model reduces to, assuming that cell $j$ has not fired before time $t$, 

$$ V(t) = \int_{0}^t l_i s(x_i-z) \exp\big(-g \cdot (t-u) \big) \mathrm{d}u $$

Thus, the probability of first spike at time $t$ is 	
$$\lambda_{i,j}(t) = \exp\left\{ \int_0^t \log\big[ 1- \psi( V_j^i(s)-V_{j,{\rm th}}) \big] \mathrm{d} s\right\} \psi\big( V_j^i(t)-V_{j,{\rm th}} \big)$$	

and the probability of no spikes up to time $t$ is  

$${\rm pr}(N_{j,i}(t)=0) =  1- \int_0^t \lambda_{i,j}(t) \mathrm{d} t = \exp\left\{ \int_0^t \log\big[ 1- \psi( V_j^i(s)-V_{j,{\rm th}}) \big] \mathrm{d} s\right\}$$

## Inference

For inference, we run gp3's mean-field stochastic variational inference. Note that we assume we have optimized the kernel parameters ahead of time, and we have a good estimate (online kernel parameter optimization is in the works).

In [3]:
mu = np.ones(X.shape[0])*5
inf_svi = MFSVI(rbf, inv_softmax(np.array([40., 1.])), LIFLike(ts = y_part, l=np.ones(len(y_part))), X, y_part, mu = mu, obs_idx = obs_idx)
inf_svi.run(2000, n_samples = 1)

From the SVI inference, we get an estimate of $s$, as well as the posterior variance. We could then use this posterior variance to select the next set of points to stimulate.

In [4]:
trace_svipred = go.Scatter3d(x = X[:,0], y = X[:,1], z=inf_svi.predict(), mode = 'markers', marker=dict(size = 2, color = color), name = "svi prediction")
trace_svivar = go.Scatter3d(x = X[:,0], y = X[:,1], z=np.exp(inf_svi.q_S), mode = 'markers', marker=dict(size = 2, color = color), name = "svi variances")
fig = tools.make_subplots(rows=1, cols=3, specs=[[{'is_3d': True}, {'is_3d': True}, {'is_3d': True}]])
fig.append_trace(trace_func, 1, 1)
fig.append_trace(trace_svipred, 1, 2)
fig.append_trace(trace_svivar, 1, 3)
iplot(fig)

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]  [ (1,3) scene3 ]



## Optimal Experimental Design

Here, we use gp3's inference to implement and evaluate a strategy for iterative experimental design. 

Consider an "experiment" to be the following: an experimenter picks a set of locations to stimulate and observes a response in the form of spike timings (as we have above). A "trial" consists of $T$ experiments: at each step $t$, we select a set of locations to stimulate, observe responses, and infer a distribution over functions.

Below, I implement two strategies for iterative experimental design. At each experiment in the trial, we select exactly one grid location to stimulate. With the "random" strategy, this location is selected at random. With the "optimal" strategy, we select the location with the highest conditional entropy of the LIF response

$$H(y|s) = \int_{S, Y} p(s|\mathcal{D}) p(y | s) \log p (y | s) ds dy$$

To estimate this entropy term, we use a Monte Carlo estimate with the following procedure:

- Sample $s$ from current variational posterior distribution $q(s;\lambda)$
- Sample $y$ conditional on $s$ from the LIF likelihood
- Calculate $\log p(y)$ using the LIF likelihood.

For both strategies, we keep track of the distance to the "true" function, and the entropy of the LIF response.

In [5]:
def update_inf(inf, choice_strat, variances, distances, lif_sim, f):
    max_idx = choice_strat(inf, lif_sim)
    response = lif_gen.sim(np.array([f[max_idx]]))
    inf_svi.obs_idx = np.append(inf.obs_idx, max_idx)
    inf_svi.y = np.append(inf.y, response)
    inf_svi.likelihood = LIFLike(ts = inf.y, l=np.ones(len(inf.y)))
    inf_svi.likelihood_opt = egrad(inf.likelihood.log_like)
    inf_svi.run(100)
    ent_norm = np.linalg.norm(np.exp(inf.q_S))
    dist_norm = np.linalg.norm(inf.q_mu - f)
    p.set_description("entropy: " + '{0:.2f}'.format(ent_norm) +
                              " | distance " + '{0:.2f}'.format(dist_norm))
    variances.append(ent_norm)
    distances.append(dist_norm)
    
    return inf, variances, distances


def rand_strat(inf, sim):
    
    return np.random.randint(0, inf_svi.n, 1)

def entropy(inf, lif_sim, n):
    
    ent = 0.
    
    for i in range(n):
        s_sample = inf.sample_post()
        t_sample = lif_sim.sim(s_sample)
        lif_like = LIFLike(ts = t_sample, l=np.ones(len(s_sample)))
        ent = ent - lif_like.log_like(s_sample, t_sample)
    
    return ent/n
    
def opt_strat(inf, lif_sim):
    
    e = entropy(inf, lif_sim, 5)
    return np.argmax(e)

Here we run the trials (set n_trials to desired number of trials). For each trial, we simulate a random cell shape and run each iterative experimental design procedure. We start off by stimulating the minimal number of locations to be able to construct a full grid (10 locations for a 10x10x10 grid).

In [6]:
from autograd import elementwise_grad as egrad

n_trials = 1
n_its = 200
all_vars = []
all_dists = []
N = tqdm_notebook(xrange(n_trials), leave=False)

for n in N:
    
    ## Generating random cell shape
    X = sim.sim_X_equispaced(D = 2, N_dim = 20, lower=0, upper=100)
    f = sim.sim_f(X, rbf, inv_softmax(np.array([40., 1.])),
              mu = 7.) - 1e-3*np.sum(np.square(X-50), 1)
    lif_gen = LIFSim(l=2)
    spikes = lif_gen.sim(f)
    X_part, y_part = sim.min_grid(X, spikes)
    X_full, y_full, obs_idx, imag_idx = sim.fill_grid(X_part, y_part)
    variances = []
    distances = []

    ## Random strategy
    inf_svi = MFSVI(rbf, inv_softmax(np.array([40., 1.])),
                    LIFLike(ts = y_part, l=np.ones(len(y_part))), X, y_part, mu = mu,
                    obs_idx = obs_idx)
    p = tqdm_notebook(xrange(n_its), leave=False)
    for k in p:
        inf_svi, variances, distances = update_inf(inf_svi, rand_strat,
                                                   variances, distances, lif_gen, f)

    ## Entropy based strategy
    inf_svi = MFSVI(rbf, inv_softmax(np.array([40., 1.])),
                    LIFLike(ts = y_part, l=np.ones(len(y_part))), X, y_part, mu = mu,
                    obs_idx = obs_idx)
    p = tqdm_notebook(xrange(n_its), leave=False)
    for k in p:
        inf_svi, variances, distances = update_inf(inf_svi, opt_strat,
                                                   variances, distances, lif_gen, f)

    all_vars.append(variances)
    all_dists.append(distances)

For each strategy, we plot the response entropy and distance to the true shape as a function of iteration.

In [7]:
variances = np.mean(np.array(all_vars), axis = 0)
distances = np.mean(np.array(all_dists), axis = 0)

iplot([go.Scatter(x = range(n_its), y = variances[:n_its], name = "rand"),
       go.Scatter(x = range(n_its), y = variances[n_its:], name = "opt")])
iplot([go.Scatter(x = range(n_its), y = distances[:n_its], name = "rand"),
       go.Scatter(x = range(n_its), y = distances[n_its:], name = "opt")])